In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd "/content/drive/MyDrive/ML/Llama2"
# !git clone https://github.com/tloen/alpaca-lora.git

/content/drive/MyDrive/ML/Llama2


### エラー対策
/alpaca-lora/finetune.pyの264-271の
```python
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))
if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)
```
の部分をコメントアウトする。

In [3]:
%cd alpaca-lora/

/content/drive/MyDrive/ML/Llama2/alpaca-lora


In [4]:
%ls

alpaca_data_1k.json               export_hf_checkpoint.py          pyproject.toml
alpaca_data_cleaned_archive.json  export_state_dict_checkpoint.py  README.md
alpaca_data_gpt4.json             finetune.py                      requirements.txt
alpaca_data.json                  generate.py                      templates/
DATA_LICENSE                      lengths.ipynb                    utils/
docker-compose.yml                LICENSE
Dockerfile                        lora-alpaca-1k/


In [5]:
%pip install -r requirements.txt

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-pgel3lbk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-pgel3lbk
  Resolved https://github.com/huggingface/peft.git to commit 7e84dec20b3106bdd0a90ba8e80187f0aec835b7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
!head alpaca_data.json

[
    {
        "instruction": "Give three tips for staying healthy.",
        "input": "",
        "output": "1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule."
    },
    {
        "instruction": "What are the three primary colors?",
        "input": "",
        "output": "The three primary colors are red, blue, and yellow."


In [7]:
import pandas as pd
df = pd.read_json("alpaca_data.json")
df

,instruction,input,output
0,Give three tips for staying healthy.,,1.Eat a balanced diet and make sure to include...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."
2,Describe the structure of an atom.,,"An atom is made up of a nucleus, which contain..."
3,How can we reduce air pollution?,,There are a number of ways to reduce air pollu...
4,Describe a time when you had to make a difficu...,,I had to make a difficult decision when I was ...
...,...,...,...
51997,Generate an example of what a resume should li...,,"Jean Tremaine\n1234 Main Street, Anytown, CA 9..."
51998,Arrange the items given below in the order to ...,"cake, me, eating",I eating cake.
51999,Write an introductory paragraph about a famous...,Michelle Obama,Michelle Obama is an inspirational woman who r...
52000,Generate a list of five things one should keep...,,1. Research potential opportunities and carefu...


In [8]:
# df.to_csv('/content/drive/MyDrive/ML/Llama2/data.csv')

In [9]:
dataset_df_1k = df[:1000]

In [10]:
dataset_df_1k.to_json('alpaca_data_1k.json', orient='records')

In [11]:
%ls

alpaca_data_1k.json               export_hf_checkpoint.py          pyproject.toml
alpaca_data_cleaned_archive.json  export_state_dict_checkpoint.py  README.md
alpaca_data_gpt4.json             finetune.py                      requirements.txt
alpaca_data.json                  generate.py                      templates/
DATA_LICENSE                      lengths.ipynb                    utils/
docker-compose.yml                LICENSE
Dockerfile                        lora-alpaca-1k/


In [12]:
!python finetune.py \
    --base_model 'openlm-research/open_llama_3b_v2' \
    --data_path './alpaca_data_1k.json' \
    --output_dir './lora-alpaca-1k' \
    --batch_size 16 \
    --micro_batch_size 16 \
    --num_epochs 4 \
    --learning_rate 1e-4 \
    --cutoff_len 512 \
    --val_set_size 900 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length

Training Alpaca-LoRA model with params:
base_model: openlm-research/open_llama_3b_v2
data_path: ./alpaca_data_1k.json
output_dir: ./lora-alpaca-1k
batch_size: 16
micro_batch_size: 16
num_epochs: 4
learning_rate: 0.0001
cutoff_len: 512
val_set_size: 900
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
add_eos_token: False
group_by_length: True
wandb_project: 
wandb_run_name: 
wandb_watch: 
wandb_log_model: 
resume_from_checkpoint: False
prompt template: alpaca

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage dire

In [13]:
%ls

alpaca_data_1k.json               export_hf_checkpoint.py          pyproject.toml
alpaca_data_cleaned_archive.json  export_state_dict_checkpoint.py  README.md
alpaca_data_gpt4.json             finetune.py                      requirements.txt
alpaca_data.json                  generate.py                      templates/
DATA_LICENSE                      lengths.ipynb                    utils/
docker-compose.yml                LICENSE
Dockerfile                        lora-alpaca-1k/


In [14]:
# !pip install gradio==3.50

In [15]:
# !python generate.py \
#     --base_model 'openlm-research/open_llama_3b_v2' \
#     --lora_weights 'lora-alpaca-1k' \
#     --share_gradio True

In [16]:
%ls

alpaca_data_1k.json               export_hf_checkpoint.py          pyproject.toml
alpaca_data_cleaned_archive.json  export_state_dict_checkpoint.py  README.md
alpaca_data_gpt4.json             finetune.py                      requirements.txt
alpaca_data.json                  generate.py                      templates/
DATA_LICENSE                      lengths.ipynb                    utils/
docker-compose.yml                LICENSE
Dockerfile                        lora-alpaca-1k/


In [17]:
import torch
from peft import PeftModel
import transformers
import gradio as gr

assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

In [18]:
tokenizer = LlamaTokenizer.from_pretrained("openlm-research/open_llama_3b_v2")

model = LlamaForCausalLM.from_pretrained(
    "openlm-research/open_llama_3b_v2",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The `load_

In [19]:
model = PeftModel.from_pretrained(
    model, "/content/drive/MyDrive/ML/Llama2/alpaca-lora/lora-alpaca-1k/", torch_dtype=torch.float16
)

In [20]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [21]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 3200, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=3200, out_features=3200, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3200, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3200, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear8bitLt(in_features=3200, out_featur

In [22]:
def evaluate(
    instruction,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=1,
    input=None,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()

In [25]:
evaluate("What are the three primary colors?")

'Red, Blue, and Green\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n### Task:\n\n##'